# DigitRecognizer


In [ ]:
import time
from datetime import datetime

#measure notebook running time
start_time = time.time()

%matplotlib inline

# backbone
import os, warnings
import numpy as np 
from numpy.random import seed
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns

# DNN
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization
from keras import regularizers, models, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import metrics
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

#from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomHeight,RandomWidth,RandomTranslation,RandomContrast,RandomZoom
import tensorflow as tf


from sklearn.model_selection import train_test_split

sns.set(style='white', context='notebook', palette='deep')
warnings.simplefilter('ignore')
print("loaded ...")

In [ ]:
TRAIN = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

#quadruple set,since I am using random preprocessing
TRAIN = pd.concat([TRAIN, TRAIN, TRAIN, TRAIN])
#5x
#TRAIN = pd.concat([TRAIN, TRAIN, TRAIN, TRAIN, TRAIN])
TRAIN.shape

In [ ]:
labels = to_categorical(TRAIN.label, num_classes=10)
TRAIN = TRAIN.drop('label', axis=1)
TRAIN = TRAIN.to_numpy() / 255

In [ ]:
TEST = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
TEST = TEST.to_numpy() / 255

In [ ]:
TRAIN = TRAIN.reshape(-1, 28, 28,1)
TEST = TEST.reshape(-1, 28, 28,1)

In [ ]:
plt.imshow(TEST[3])
plt.show()

## Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(TRAIN, labels, test_size = 0.3, random_state = 13, stratify=labels)

# DNN

In [ ]:
def plot_loss(loss,val_loss):
    plt.figure()
    plt.plot(loss)
    plt.plot(val_loss)
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    plt.show()

def plot_accuracy(acc,val_acc):
    plt.figure()
    plt.plot(acc)
    plt.plot(val_acc)
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show() 

In [ ]:
# Reproducibility
RANDOM_SEED = 1313
def set_seed(sd=13):
    seed(sd)
    np.random.seed(sd)
    tf.random.set_seed(sd)
    os.environ['PYTHONHASHSEED'] = str(sd)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(RANDOM_SEED)

In [ ]:
DNN = Sequential()
DNN.add(Input(shape = (28,28,1), name = 'input'))

#preproc
DNN.add(RandomContrast(factor=0.15, seed=RANDOM_SEED))
DNN.add(RandomRotation(factor=0.05, seed=RANDOM_SEED))
DNN.add(RandomZoom(height_factor=0.02,width_factor=0.02, seed=RANDOM_SEED))
##

DNN.add(Conv2D(64, kernel_size= (3,3), activation = 'relu', padding="same")) 
DNN.add(BatchNormalization())
DNN.add(MaxPool2D(2))
DNN.add(Dropout(0.2, seed = RANDOM_SEED)) 

DNN.add(Conv2D(128, kernel_size= (3,3), activation = 'relu', padding="same"))
DNN.add(BatchNormalization())
DNN.add(MaxPool2D(2))
DNN.add(Dropout(0.2, seed = RANDOM_SEED)) 

DNN.add(Conv2D(256, kernel_size= (3,3), activation = 'relu', padding="same"))
DNN.add(BatchNormalization())
DNN.add(MaxPool2D(2))
DNN.add(Dropout(0.2, seed = RANDOM_SEED))

DNN.add(Flatten())

DNN.add(Dense(256, activation = 'relu'))
DNN.add(Dropout(0.1, seed = RANDOM_SEED))

DNN.add(Dense(64, activation = 'relu'))
DNN.add(BatchNormalization())

DNN.add(Dense(10, activation = 'softmax', name = "output"))
DNN.summary()

In [ ]:
%%time
DNN.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-03), metrics=['accuracy'])
early_stopping_monitor = EarlyStopping(patience=5, monitor='val_accuracy')
checkpoint = ModelCheckpoint("weights.hdf5", monitor = 'val_accuracy', save_best_only = True)
DNN.fit(X_train,y_train, validation_data=(X_test,y_test), callbacks=[early_stopping_monitor, checkpoint], epochs=200, batch_size=256, verbose=0);
DNN.load_weights("weights.hdf5")

In [ ]:
%%time
plot_loss(DNN.history.history['loss'], DNN.history.history['val_loss'])
plot_accuracy(DNN.history.history['accuracy'], DNN.history.history['val_accuracy'])

_, train_dnn_accuracy = DNN.evaluate(X_train, y_train)
_, dnn_accuracy = DNN.evaluate(X_test, y_test)
print('Train accuracy: {:.2f} %'.format(train_dnn_accuracy*100))
print('Accuracy: {:.2f} %'.format(dnn_accuracy*100))
print('Overfit: {:.2f} % '.format((train_dnn_accuracy - dnn_accuracy)*100))

# Predict

In [ ]:
results = DNN.predict(TEST)
predictions = pd.Series(np.argmax(results, axis = 1),name="Label")

# Submission

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)
submission.head(10)

In [ ]:
submission.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

In [ ]:
end_time = time.time()
print("Notebook run time: {:.1f} seconds. Finished at {}".format(end_time - start_time, datetime.now()) )